In [1]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
from model import TextGenerator 
from get_data import *

In [2]:
#preparing raw data
filename = "alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
seq_len = 50
n_chars, n_vocab, n_examples, X, Y, int_to_char, char_to_int = get_data(raw_text, seq_len)

In [12]:
#create model
H = 512 #hidden size
D = n_vocab # features
layers = 2 # number of lsmt layers
B = 16 # batch size
V = n_vocab # vocabulary size
model = TextGenerator(D,H,layers,B,V)
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.08, momentum = 0.9, nesterov = True)
print (model)
loss_function = nn.CrossEntropyLoss()

TextGenerator (
  (lstm): LSTM(47, 512, num_layers=2, batch_first=True)
  (dropout): Dropout (p = 0.5)
  (scores): Linear (512 -> 47)
  (batchnormD): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
)


In [ ]:
num_iter = n_examples/B

for epoch in range(1):
    hidden = model.init_hidden(B)
    for it in xrange(num_iter):
        model.zero_grad()

        start = np.random.randint(0, len(X)-2-B)
        inp = prepeare_X(X[start:start+B],B,seq_len, n_vocab)
        inp = inp.cuda() 
        
        target = Y[start:start+B]
        tensor1 = torch.LongTensor(target)
        m = autograd.Variable(tensor1)
        m = m.cuda()
        
        tag_scores, hidden = model(inp, hidden)
        tag_scores = tag_scores.view(B*seq_len, -1)
        m = m.view(B*seq_len)
        loss = loss_function(tag_scores, m)
        if ( it%100 == 0 ):
            print (epoch, it , loss)
        hidden[0].detach_()
        hidden[1].detach_()
        loss.backward(retain_variables=True)
        optimizer.step()
        target = Y[start+(epoch+1)]
        pattern = X[start+(epoch+1)]

0 0 Variable containing:
 3.8975
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 100 Variable containing:
 3.1021
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 200 Variable containing:
 2.7777
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [9]:
start = np.random.randint(0, len(X)-2)
pattern = X[start]
s = ""
hidden = model.init_hidden(1)
for it in xrange(1000):

    inp = prepeare_X(pattern,1,seq_len, n_vocab)
    scores, hidden = model.forward2(inp.cuda(), hidden) #for GPU
    hidden[0].detach_()
    hidden[1].detach_()
#    scores, hidden = model.forward2(inp, hidden)
    qwer, tmp = torch.max(scores,1)
    final = np.squeeze(tmp.data.cpu().numpy()) #for GPU
#    final = np.squeeze(tmp.data.numpy())
    predicted_char = int_to_char[final[seq_len-1]]
    pattern.append(final[seq_len-1])
    pattern = pattern[1:]
    s += predicted_char

In [10]:
print (s)

uthe soull ce the wathe sout in the be the woull the be the wathe south the wall the wer the she the the she the the woull the the the sed the sout the the wall the the she the sout the she the the sed the the the sout the the se watling the sound the sed the the she the s and the she the found the sout fout the she fout the she the sound the the she fout and the waing the she fout on the waind the be the waling the wand the she fout on the wall the the wall the be the she fout nould the wall the sound the sound the the soull the sed the wall the the she the the the sout and the sout of the she the s mound the she the wand the she the s bot on the she found the sout of the the sout the she sound the the saing the she sould and the se the southe sou fout alice sout an the she and the sound alit allice and the wall the wall the se and the seat ou the she waing the she wall the fout on the seat on the sou the walit on the me the sout ou the was in the the the the wat the so the the wou th